# Hitchhiker's Guide to .NET for Apache Spark

Welcome to the .NET for Apache Spark tutorial! We are glad to have you here. Before we begin, let us cover answers to a few quick questions:

 - #### What is .NET for Apache Spark?
  .NET for Apache Spark provides high performance APIs for using Apache Spark from C# and F#. With these .NET APIs, you can access the most popular Dataframe and SparkSQL aspects of Apache Spark, for working with structured data, and Spark Structured Streaming, for working with streaming data.

  .NET for Apache Spark is compliant with .NET Standard - a formal specification of .NET APIs that are common across .NET implementations. This means you can use .NET for Apache Spark anywhere you write .NET code allowing you to reuse all the knowledge, skills, code, and libraries you already have as a .NET developer.

 - #### Where can I find more on .NET for Apache Spark?
  https://github.com/dotnet/spark

 - #### I did not know there was a REPL for C#!?
   Great question! :) We collaborated with the .NET team and they built one for us! https://github.com/dotnet/interactive 

Whew! Now that we have covered some basic information, let's begin! 

Since the .NET REPL is something very new, let us start by exploring what you can do with the REPL. 

# Basic Capabilities of the C# REPL

In [4]:
// Simple assignments should just work 
var x = 1 + 25;

In [5]:
// You can either use traditional approach to printing a variable...
Console.WriteLine(x);

// ... or just type it and execute a cell
256

26
256

In [6]:
// You could write code that throws exceptions and they bubble up to the notebook
throw new Exception("watzzz");

In [7]:
// Whatever code is deemed invalid by the C# Compiler, is invalid here too 
var z = 12345

In [8]:
// You can even play with built-in libraries/functions
Enumerable.Range(1, 5)

index,value
0,1
1,2
2,3
3,4
4,5


In [9]:
// And now for some C# 8.0 features. If you haven't read it,
// here's the link: 
// https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-8
1..4

Start,End
"{ Index: Value: 1, IsFromEnd: False }","{ Index: Value: 4, IsFromEnd: False }"


In [10]:
// We can even do pattern matching!
public static string RockPaperScissors(string first, string second)
    => (first, second) switch
    {
        ("rock", "paper") => "rock is covered by paper. Paper wins.", // <-- Next cell prints this out
        ("rock", "scissors") => "rock breaks scissors. Rock wins.",
        ("paper", "rock") => "paper covers rock. Paper wins.",
        ("paper", "scissors") => "paper is cut by scissors. Scissors wins.",
        ("scissors", "rock") => "scissors is broken by rock. Rock wins.",
        ("scissors", "paper") => "scissors cuts paper. Scissors wins.",
        (_, _) => "tie"
    };

In [11]:
RockPaperScissors("rock", "paper")

rock is covered by paper. Paper wins.

In [12]:
// Now, for the fun part! You can render HTML
display(
    div(
        h1("Our Incredibly Declarative Example"),
        p("Can you believe we wrote this ", b("in C#"), "?"),
        img[src:"https://media.giphy.com/media/xUPGcguWZHRC2HyBRS/giphy.gif"],
        p("What will ", b("you"), " create next?")
    )
);

Our Incredibly Declarative Example Can you believe we wrote this in C# ? What will you create next?

# Looking at data through Spark.NET


In [38]:
// Let us use some sample data. In this cell, we create this data 
// from *scratch* but you can also load it from your storage container. 
// For instance, 
// var df = spark.Read().Json("wasbs://<account>@<container>.blob.core.windows.net/people.json");

using Microsoft.Spark.Sql;
using Microsoft.Spark.Sql.Types;
using static Microsoft.Spark.Sql.Functions;

var schema = new StructType(new List<StructField>()
    {
        new StructField("id", new IntegerType()),
        new StructField("name", new StringType())
    });

var data = new List<GenericRow>();
data.Add(new GenericRow(new object[] { 0,  "Michael" }));
data.Add(new GenericRow(new object[] { 1,  "Elva"    }));
data.Add(new GenericRow(new object[] { 2,  "Terry"   }));
data.Add(new GenericRow(new object[] { 3,  "Steve"   }));
data.Add(new GenericRow(new object[] { 4,  "Brigit"  }));
data.Add(new GenericRow(new object[] { 5,  "Niharika"}));
data.Add(new GenericRow(new object[] { 6,  "Rahul"   }));
data.Add(new GenericRow(new object[] { 7,  "Tomas"   }));
data.Add(new GenericRow(new object[] { 8,  "Euan"   }));
data.Add(new GenericRow(new object[] { 9,  "Lev"   }));
data.Add(new GenericRow(new object[] { 10, "Saveen"   }));

var df = spark.CreateDataFrame(data, schema);
df.Show();

+---+--------+
| id|    name|
+---+--------+
|  0| Michael|
|  1|    Elva|
|  2|   Terry|
|  3|   Steve|
|  4|  Brigit|
|  5|Niharika|
|  6|   Rahul|
|  7|   Tomas|
|  8|    Euan|
|  9|     Lev|
| 10|  Saveen|
+---+--------+

In [39]:
// Wait, that rendering is old-school plain! Let's spice things up a bit!
// What we're doing here is to define a specific formatter that is tied to 
// Microsoft.Spark.Sql.DataFrame and registering it. When we then invoke
// display() and pass a DataFrame, the formatter is invoked, which then
// generates the necessary HTML

Microsoft.DotNet.Interactive.Formatting.Formatter<Microsoft.Spark.Sql.DataFrame>.Register((df, writer) =>
{
    var headers = new List<dynamic>();
    var columnNames = df.Columns();
    headers.Add(th(i("index")));
    headers.AddRange(columnNames.Select(c => th(c)));

    var rows = new List<List<dynamic>>();
    var currentRow = 0;
    var dfRows = df.Take(Math.Min(20, (int)df.Count()));
    foreach (Row dfRow in dfRows)
    {
        var cells = new List<dynamic>();
        cells.Add(td(currentRow));

        foreach (string columnName in columnNames)
        {
            cells.Add(td(dfRow.Get(columnName)));
        }

        rows.Add(cells);
        ++currentRow;
    }

    var t = table[@border: "0.1"](
        thead[@style: "background-color: blue; color: white"](headers),
        tbody[@style: "color: red"](rows.Select(r => tr(r))));

    writer.Write(t);
}, "text/html");

In [40]:
// Now, let's try rendering the Spark's DataFrame in two ways...

// ... a regular way ...
df.Show();

// ... and just typing df (so it invokes the formatter we just defined)
df

0,0,Michael
1,1,Elva
2,2,Terry
3,3,Steve
4,4,Brigit
5,5,Niharika
6,6,Rahul
7,7,Tomas
8,8,Euan
9,9,Lev
10,10,Saveen


In [41]:
// Let us now try something more advanced like, defining C# classes on-the-fly...
public static class A {
    public static readonly string s = "The person named ";
}

In [45]:
// ... and just for illustration, let's define one more simple class
public static class B {
    private static Random _r = new Random();
    private static List<string> _moods = new List<string>{ "happy","funny","awesome","cool"};

    public static string GetMood() {
        return _moods[_r.Next(_moods.Count)];
    }
}

In [50]:
// Let us now define a Spark User-defined Function (UDF) that utilizes
// the classes we just defined above. If you do not recognize the syntax
// below, here's some relevant documentation:
// https://docs.microsoft.com/en-us/dotnet/api/system.func-2?view=netframework-4.8
// https://github.com/dotnet/spark/blob/master/examples/Microsoft.Spark.CSharp.Examples/Sql/Basic.cs
//
// Note: If you change the class definition above, and execute the cell,
// you should re-execute this cell (i.e., the cell that defines the UDF)
var udf = Udf<string, string>(str => $"{A.s} - {str} - is {B.GetMood()}!");

In [51]:
// Let's use the UDF on our Spark DataFrame
display(
    df
    .Select(
        udf((Microsoft.Spark.Sql.Column)df["name"])));

0,The person named - Michael - is happy!
1,The person named - Elva - is happy!
2,The person named - Terry - is funny!
3,The person named - Steve - is awesome!
4,The person named - Brigit - is cool!
5,The person named - Niharika - is happy!
6,The person named - Rahul - is cool!
7,The person named - Tomas - is cool!
8,The person named - Euan - is cool!
9,The person named - Lev - is funny!
10,The person named - Saveen - is awesome!


In [52]:
// Tables are not that interesting, right? :) Let's do some visualizations now.
// Let us start with something simple to illustrate the idea. We highly encourage
// you to look at https://fslab.org/XPlot/ to understand how you can use XPlot's
// full capabilities. While the examples are in F#, it is fairly straightforward
// to rewrite in C#.

using XPlot.Plotly;

var lineChart = Chart.Line(new List<int> { 1, 2, 3, 4, 5, 6, 10, 44 });
lineChart.WithTitle("My awesome chart");
lineChart.WithXTitle("X axis");
lineChart.WithYTitle("Y axis");
display(lineChart);

In [54]:
// Good! Now let us try to visualize the Spark DataFrame we have.
// Now is a good time to refresh your concept of a Spark DataFrame
// https://spark.apache.org/docs/latest/sql-programming-guide.html
// Remember that a Spark DataFrame is a distributed representation 
// of your dataset (yes, even if your data is a few KB). Since we
// are using a visualization library, we need to first 'collect'
// (notice how we are using df.Collect().ToArray() below)
// all the data that is distributed on your cluster, and shape it
// appropriately for XPlot.
//
// Note: Visualizations are good for smaller datasets (typically, 
// a few 10s of thousands of data points coming to KBs), so if you are
// trying to visualize GBs of data, it is usually a good idea to
// summarize your data appropriately using Spark.NET's APIs. For
// a list of summarization APIs, see here:
// https://docs.microsoft.com/en-us/dotnet/api/microsoft.spark.sql.functions?view=spark-dotnet

var names = new List<string>();
var ids = new List<int>();

foreach (Row row in df.Collect().ToArray())
{
 names.Add(row.GetAs<string>("name"));
 int? id = row.GetAs<int?>("id");
 ids.Add( id ?? 0);
}
var bar = new Graph.Bar
{
 name = "bar chart",
 x = names,
 y = ids
};

var chart = Chart.Plot(new[] {bar});
display(chart);

In [75]:
// As a final step, let us now plot a histogram of a random dataset

using XPlot.Plotly;

var schema = new StructType(new List<StructField>()
    {
        new StructField("number", new DoubleType())
    });

Random random = new Random();

var data = new List<GenericRow>();
for(int i = 0; i <=100; i++) {
    data.Add(new GenericRow(new object[] { random.NextDouble() }));
}

var histogramDf = spark.CreateDataFrame(data, schema);
histogramDf.Show()

+--------------------+
|              number|
+--------------------+
|  0.9696137248396938|
|0.032666774947506735|
|  0.9982835552647168|
|   0.623668246261621|
|   0.886883255507277|
|  0.6830984720415895|
|   0.208581972964379|
| 0.11822117544627803|
| 0.41178633617786053|
| 0.39932316653398947|
| 0.07784702818740487|
|   0.717773464842594|
| 0.21077453215176917|
|0.008377783935692992|
| 0.16170730356206525|
| 0.46337959564448317|
|  0.7769349020798387|
| 0.15755635414158756|
|  0.9235577191801545|
|  0.9462093380029357|
+--------------------+
only showing top 20 rows

In [76]:
// Time to use LINQ (or Language Integrated Query) :)
// For those that are not familiar with LINQ, you can read more about it
// here: https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/linq/

using System.Linq;

// Let us take the histogramDf we loaded through Spark and sample some data points
// for the histogram. We will then use LINQ to shape the data for our next 
// steps (visualization!)
var sample1 = 
        histogramDf.Sample(0.5, true).Collect().ToArray() // <---- Spark APIs
        .Select(x => x.GetAs<double>("number")); // <---- LINQ APIs
        
// Let us create two more sample sets we can use for plotting
var sample2 = histogramDf.Sample(0.3, false).Collect().ToArray().Select(x => x.GetAs<double>("number"));
var sample3 = histogramDf.Sample(0.6, true).Collect().ToArray().Select(x => x.GetAs<double>("number"));

In [77]:
// Let us plot the histograms now!
var hist1 = new Graph.Histogram{x = sample1, opacity = 0.75};
var hist2 = new Graph.Histogram{x = sample2, opacity = 0.75};
var hist3 = new Graph.Histogram{x = sample3, opacity = 0.75};

In [78]:
Chart.Plot(new[] {hist1})

In [79]:
Chart.Plot(new[] {hist2})

In [80]:
Chart.Plot(new[] {hist3})

In [81]:
// but wait, that's three different graphs and it's impossible to read them
// altogether! Let's try an overlay histogram, shall we?
var layout = new XPlot.Plotly.Layout.Layout{barmode="overlay", title="Overlaid Histogram"};
var histogram = Chart.Plot(new[] {hist1, hist2, hist3});
histogram.WithLayout(layout);
histogram

In [86]:
// And for the final touches
using static XPlot.Plotly.Graph;

layout.title = "Overlaid Histogram with cool colors!";
hist1.marker = new Marker {color = "#D65108)"};
hist2.marker = new Marker {color = "#ffff00"}; 
hist3.marker = new Marker {color = "#462255"};

histogram